In [28]:
import subprocess, re, sys, os, tqdm, shutils
from shutil import copyfile

In [2]:
FFMPEG_PATH = "ffmpeg-N-99973-g0066bf4d1a-win64-gpl-shared-vulkan\\bin\\ffmpeg.exe"

THRESHOLD = -50

DURATION = 0.4

#### Let's first test one file

In [3]:
TEST_VIDEO = "ravdess\\video\\Actor_01\\01-01-01-01-01-01-01.mp4"

In [4]:
# ffmpeg-N-99973-g0066bf4d1a-win64-gpl-shared-vulkan\bin\ffmpeg.exe -i ravdess\audio\Actor_01\03-01-01-01-01-01-01.wav 
# -af silencedetect=n=-50dB:d=0.5 -f null - 2> sdout.txt

subprocess.call(FFMPEG_PATH+' -i '+TEST_VIDEO+' -af silencedetect=n=' + str(THRESHOLD) + 'dB:d=' + \
                    str(DURATION) + ' -f null - 2> sdout.txt', shell=True)

0

In [5]:
with open("sdout.txt") as f:
    out_txt = f.read()

In [6]:
start_times = []
end_times = []

In [7]:
for line in out_txt.split("\n"):
    if "silence_start" in line: start_times.append(re.search('.*?:\s([\d\.]*)', line).group(1))
    if "silence_end" in line: end_times.append(re.search('.*?:\s([\d\.]*)', line).group(1))

In [8]:
start_times

['0', '2.26673']

In [9]:
end_times

['0.858375', '3.30667']

In [10]:
# ffmpeg-N-99973-g0066bf4d1a-win64-gpl-shared-vulkan\bin\ffmpeg.exe -i 
# ravdess\video\Actor_01\01-01-01-01-01-01-01.mp4 -ss 0.858375 -t 2.26673 -async 1 cut.mp4

subprocess.call(FFMPEG_PATH+' -i '+TEST_VIDEO+' -ss '+end_times[0]+' -t '+start_times[1]+' -async 1 cut.mp4', shell=True)

0

In [11]:
os.remove("cut.mp4")
os.remove("sdout.txt")

#### Now, for the whole dataset

In [32]:
audio_root = "./ravdess/audio"
video_root = "./ravdess/video"


for root, dirs, files in tqdm.tqdm(os.walk(audio_root, topdown=False)):
    for file in files:
        subprocess.call(FFMPEG_PATH+' -i '+os.path.join(root,file)+' -af silencedetect=n=' + str(THRESHOLD) + 'dB:d=' + \
                    str(DURATION) + ' -f null - 2> sdout.txt', shell=True)
        with open("sdout.txt") as f:
            out_txt = f.read()
        start_times = []
        end_times = []
        
        if not os.path.exists(root.replace("audio", "audio_trim")):
            os.makedirs(root.replace("audio", "audio_trim"))
        
        
        for line in out_txt.split("\n"):
            if "silence_start" in line: start_times.append(re.search('.*?:\s([\d\.]*)', line).group(1))
            if "silence_end" in line: end_times.append(re.search('.*?:\s([\d\.]*)', line).group(1))
                
        if len(start_times) != 2:
            copyfile(os.path.join(root,file), os.path.join(root.replace("audio", "audio_trim"),file))
        else:
            start = end_times[0]
            end = start_times[-1]
            
            subprocess.call(FFMPEG_PATH+' -i '+os.path.join(root,file)+' -ss '+start+' -t '+end+ \
                        ' -async 1 '+os.path.join(root.replace("audio", "audio_trim"),file), shell=True)

25it [05:38, 13.54s/it]


In [33]:
audio_root = "./ravdess/audio"
video_root = "./ravdess/video"


for root, dirs, files in tqdm.tqdm(os.walk(video_root, topdown=False)):
    for file in files:
        subprocess.call(FFMPEG_PATH+' -i '+os.path.join(root,file)+' -af silencedetect=n=' + str(THRESHOLD) + 'dB:d=' + \
                    str(DURATION) + ' -f null - 2> sdout.txt', shell=True)
        with open("sdout.txt") as f:
            out_txt = f.read()
        start_times = []
        end_times = []
        
        if not os.path.exists(root.replace("video", "video_trim")):
            os.makedirs(root.replace("video", "video_trim"))
        
        
        for line in out_txt.split("\n"):
            if "silence_start" in line: start_times.append(re.search('.*?:\s([\d\.]*)', line).group(1))
            if "silence_end" in line: end_times.append(re.search('.*?:\s([\d\.]*)', line).group(1))
                
        if len(start_times) != 2:
            copyfile(os.path.join(root,file), os.path.join(root.replace("video", "video_trim"),file))
        else:
            start = end_times[0]
            end = start_times[-1]
            
            subprocess.call(FFMPEG_PATH+' -i '+os.path.join(root,file)+' -ss '+start+' -t '+end+ \
                        ' -async 1 '+os.path.join(root.replace("video", "video_trim"),file), shell=True)

25it [37:05, 89.04s/it]
